# Transfer Learning


In [104]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

## Dataset

https://www.cs.toronto.edu/~kriz/cifar.html

In [97]:
from keras.datasets import cifar10
from keras.applications.vgg16 import preprocess_input
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()
X_train_full = X_train_full.astype('float32')

# normalisation
# X_train_full = X_train_full/255.0
# X_test = X_test/255.0

# vgg16 preprocess
X_train_full = preprocess_input(X_train_full)
X_test = preprocess_input(X_test)

# one-hot encoding
y_train_full = keras.utils.to_categorical(y_train_full, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

# get validation dataset
np.random.seed(42) # we set the random seed to make sure everytime the data is shuffled in the same way 
shuffled_indices = np.random.permutation(X_train_full.shape[0])
X_valid, X_train = X_train_full[shuffled_indices[:5000]], X_train_full[shuffled_indices[5000:]]
y_valid, y_train = y_train_full[shuffled_indices[:5000]], y_train_full[shuffled_indices[5000:]]

In [98]:
X_train.shape

(45000, 32, 32, 3)

## VGG16
![](vgg16.png)

In [110]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape=(32, 32, 3), weights='imagenet')
# vgg16_model = tf.keras.applications.vgg16.VGG16()
# when you change the input shape, keras will automatically change the shapes of the other layers accordingly . to meet the new input shape
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

## Transfer Learning

In [112]:
x = vgg16_model.layers[-5].output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256,activation='relu')(x) # dense layer 1
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(256,activation='relu')(x) # dense layer 2
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
output = keras.layers.Dense(units=10, activation='softmax')(x) # output layer with softmax activation
model = keras.Model(inputs=vgg16_model.input, outputs=output)

# freeze the layers from VGG16 
for layer in vgg16_model.layers:
    layer.trainable=False

model.summary()

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)       

In [101]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [102]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
1407/1407 [==============================] - 81s 57ms/step - loss: 0.9192 - accuracy: 0.6831 - val_loss: 0.7106 - val_accuracy: 0.7648
Epoch 2/3
1407/1407 [==============================] - 88s 63ms/step - loss: 0.7036 - accuracy: 0.7558 - val_loss: 0.6680 - val_accuracy: 0.7696
Epoch 3/3
1407/1407 [==============================] - 87s 62ms/step - loss: 0.6532 - accuracy: 0.7701 - val_loss: 0.7162 - val_accuracy: 0.7796


Freeze more layers to see whether it performs better. 

In [ ]:
model.evaluate(X_test, y_test)